In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import statistics as st
from datetime import datetime
import numpy as np
import os
from torchvision.datasets import ImageFolder
import torchvision.models as models

from torchvision.utils import make_grid

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpa")

In [3]:
print(device)

cuda


In [4]:
trans1 = transforms.Compose([transforms.RandomCrop(32,padding=4,padding_mode="reflect"),
                             transforms.RandomHorizontalFlip(p=0.5),
                             transforms.ToTensor(),
                             transforms.Normalize(mean=[0.4914,0.4822,0.4465],
                                                  std=[0.2023,0.1994,0.2010])])
trans2 = transforms.Compose([transforms.ToTensor(),
                             transforms.Normalize(mean=[0.4914,0.4822,0.4465],
                                                  std=[0.2023,0.1994,0.2010])])

In [5]:
train_cifar = torchvision.datasets.CIFAR100("Cifar100",train=True,download=True,transform=trans1)
test_cifar = torchvision.datasets.CIFAR100("Cifar100",train=False,download=True,transform=trans2)

print(len(train_cifar))
print(len(test_cifar))

Files already downloaded and verified
Files already downloaded and verified
50000
10000


In [ ]:
# n=0
# while n<100:
#     for x,y in train_cifar:
#         print(y)
#         n+=1
print(train_cifar.classes)
print(train_cifar.class_to_idx)

In [6]:
batch_size=100
train_data = DataLoader(train_cifar,batch_size=batch_size,shuffle=True,num_workers=1,pin_memory=True)
test_data = DataLoader(test_cifar,batch_size=batch_size,shuffle=False,num_workers=1,pin_memory=True)

In [ ]:
[x for x in dir(models)]

In [ ]:
mm = models.resnet50()
mm

In [7]:
def conv_block1(in_channel,out_channel):
    layers = [nn.Conv2d(in_channel,out_channel,kernel_size=3,padding=1,bias=False),
              nn.BatchNorm2d(out_channel,momentum=0.1),
              nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1,bias=False),
              nn.BatchNorm2d(out_channel,momentum=0.1),
              nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1,bias=False),
              nn.BatchNorm2d(out_channel,momentum=0.1),
              nn.ReLU(inplace=True),
              nn.Conv2d(out_channel,out_channel,kernel_size=3,padding=1,bias=False)]
    return nn.Sequential(*layers)

def batch_norm(in_channel):
    layers = [nn.BatchNorm2d(in_channel,momentum=0.1),
             nn.ReLU(inplace=True)]
    
    return nn.Sequential(*layers)
    
    

In [8]:
class resnet_x(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3,16,kernel_size=3,padding=1,bias=False)
        self.conv2 = batch_norm(16)
        self.conv2_1 = nn.Conv2d(16,96,kernel_size=1,bias=False)
        
        self.conv3 = conv_block1(16,96)
        self.conv3_2 = batch_norm(96)
        self.conv4 = conv_block1(96,96)
        self.conv4_2=batch_norm(96)
        self.conv5 = conv_block1(96,96)
        self.conv5_2 = batch_norm(96)
        self.conv6 = conv_block1(96,96)
        self.conv6_2 = batch_norm(96)
        
        self.conv7 = nn.Conv2d(96,192,kernel_size=1,bias=False)
        self.conv8 = conv_block1(96,192)
        self.conv8_2 = batch_norm(192)
        self.conv9 = conv_block1(192,192)
        self.conv9_2 = batch_norm(192)
        self.conv10 = conv_block1(192,192)
        self.conv10_2 = batch_norm(192)
        self.conv11 = conv_block1(192,192)
        self.conv11_2 = batch_norm(192)
        
        self.conv12 = nn.Conv2d(192,384,kernel_size=1,bias=False)
        self.conv13 = conv_block1(192,384)
        self.conv13_2 = batch_norm(384)
#         self.conv14 = conv_block1(384,384)
#         self.conv14_2 = batch_norm(384)
#         self.conv15 = conv_block1(384,384)
#         self.conv15_2 = batch_norm(384)
#         self.conv16 = conv_block1(384,384)
#         self.conv16_2 = batch_norm(384)
        
#         self.conv17 = nn.Conv2d(384,768,kernel_size=1,bias=False)
#         self.conv18 = conv_block1(384,768)
#         self.conv18_2 = batch_norm(768)
#         self.conv19 = conv_block1(768,768)
#         self.conv19_2 = batch_norm(768)
#         self.conv20 = conv_block1(768,768)
#         self.conv20_2 = batch_norm(768)
#         self.conv21 = conv_block1(768,768)
#         self.conv21_2 = batch_norm(768)
        
        self.linear1 = nn.AdaptiveAvgPool2d((1,1))
        self.linear2 = nn.Linear(384,100)
    
    
    def forward(self,data):
        out = self.conv1(data)
        out = self.conv2(out)
        out = self.conv2_1(out)+self.conv3(out)
        out = self.conv3_2(out)
        out = out + self.conv4(out)
        out = self.conv4_2(out)
        out = out + self.conv5(out)
        out = self.conv5_2(out)
        out = out + self.conv6(out)
        out = self.conv6_2(out)
        
        out = self.conv7(out)+self.conv8(out)
        out = self.conv8_2(out)
        out = out + self.conv9(out)
        out = self.conv9_2(out)
        out = out + self.conv10(out)
        out = self.conv10_2(out)
        out = out +self.conv11(out)
        out = self.conv11_2(out)
        
        out = self.conv12(out) + self.conv13(out)
        out = self.conv13_2(out)
#         out = out+ self.conv14(out)
#         out = self.conv14_2(out)
#         out = out + self.conv15(out)
#         out = self.conv15_2(out)
#         out = out + self.conv16(out)
#         out = self.conv16_2(out)
        
#         out = self.conv17(out) + self.conv18(out)
#         out = self.conv18_2(out)
#         out = out + self.conv19(out)
#         out = self.conv19_2(out)
#         out = out + self.conv20(out)
#         out = self.conv20_2(out)
#         out = out + self.conv21(out)
#         out = self.conv21_2(out)
        
        out = self.linear1(out)
        out = out.reshape(-1,384)
        out = self.linear2(out)
        
        return out
        
        

In [9]:
mymodel = resnet_x().to(device)

In [ ]:
# for x,y in train_data:
#     a = mymodel(x)
#     print(a.shape)
#     break

In [10]:
def fit(epoch,train,test,model,loss_fun,opt):
    
    for a in range(epoch):
        n=1
        his=[]
        for x,y in train:
            x,y = x.to(device),y.to(device)
            pred = model(x)
            __,preds = torch.max(pred,dim=1)
            acc = torch.sum(preds==y).item()/len(y)
            his.append(torch.sum(preds==y).item())
            loss = loss_fun(pred,y)
            loss.backward()
            opt.step()
            opt.zero_grad()
            n+=1
            if n%10==0:
                print("Epoch # [{}/{}], batch # {} with mini batch accuracy of {:.3%} and loss of {:.3f}"\
                      .format(a+1,epoch,n,acc,loss.item()))
        if (a+1)%1==0:
            print("Epoch # [{}/{}] with overall accuracy of {:.3%}".format(a+1,epoch,sum(his)/len(train_cifar)))
        
        with torch.no_grad():
            history=[]
            for x,y in test:
                x,y = x.to(device),y.to(device)
                p = model(x)
                __,prediction = torch.max(p,dim=1)
                result = torch.sum(prediction==y).item()
                history.append(result)
            print("Test result of epoch [{}/{}] has accuracy of {:.3%}".format(a+1,epoch,sum(history)/len(test_cifar)))
            
        
            

In [13]:
loss_fun = F.cross_entropy
opt = torch.optim.Adam(mymodel.parameters(),lr=0.00001)

In [15]:
fit(10,train_data,test_data,mymodel,loss_fun,opt)

Epoch # [1/10], batch # 10 with mini batch accuracy of 71.000% and loss of 0.959
Epoch # [1/10], batch # 20 with mini batch accuracy of 76.000% and loss of 0.916
Epoch # [1/10], batch # 30 with mini batch accuracy of 67.000% and loss of 1.177
Epoch # [1/10], batch # 40 with mini batch accuracy of 63.000% and loss of 1.218
Epoch # [1/10], batch # 50 with mini batch accuracy of 75.000% and loss of 0.937
Epoch # [1/10], batch # 60 with mini batch accuracy of 66.000% and loss of 1.225
Epoch # [1/10], batch # 70 with mini batch accuracy of 79.000% and loss of 0.822
Epoch # [1/10], batch # 80 with mini batch accuracy of 67.000% and loss of 1.135
Epoch # [1/10], batch # 90 with mini batch accuracy of 72.000% and loss of 1.015
Epoch # [1/10], batch # 100 with mini batch accuracy of 64.000% and loss of 1.095
Epoch # [1/10], batch # 110 with mini batch accuracy of 71.000% and loss of 0.947
Epoch # [1/10], batch # 120 with mini batch accuracy of 67.000% and loss of 1.023
Epoch # [1/10], batch # 1

Epoch # [2/10], batch # 490 with mini batch accuracy of 65.000% and loss of 0.991
Epoch # [2/10], batch # 500 with mini batch accuracy of 69.000% and loss of 1.101
Epoch # [2/10] with overall accuracy of 68.648%
Test result of epoch [2/10] has accuracy of 57.780%
Epoch # [3/10], batch # 10 with mini batch accuracy of 75.000% and loss of 0.936
Epoch # [3/10], batch # 20 with mini batch accuracy of 70.000% and loss of 0.846
Epoch # [3/10], batch # 30 with mini batch accuracy of 66.000% and loss of 1.098
Epoch # [3/10], batch # 40 with mini batch accuracy of 68.000% and loss of 1.126
Epoch # [3/10], batch # 50 with mini batch accuracy of 70.000% and loss of 0.980
Epoch # [3/10], batch # 60 with mini batch accuracy of 72.000% and loss of 0.935
Epoch # [3/10], batch # 70 with mini batch accuracy of 76.000% and loss of 0.970
Epoch # [3/10], batch # 80 with mini batch accuracy of 63.000% and loss of 1.111
Epoch # [3/10], batch # 90 with mini batch accuracy of 65.000% and loss of 0.942
Epoch #

Epoch # [4/10], batch # 460 with mini batch accuracy of 65.000% and loss of 1.034
Epoch # [4/10], batch # 470 with mini batch accuracy of 69.000% and loss of 1.082
Epoch # [4/10], batch # 480 with mini batch accuracy of 63.000% and loss of 1.118
Epoch # [4/10], batch # 490 with mini batch accuracy of 68.000% and loss of 1.108
Epoch # [4/10], batch # 500 with mini batch accuracy of 72.000% and loss of 0.877
Epoch # [4/10] with overall accuracy of 69.500%
Test result of epoch [4/10] has accuracy of 57.840%
Epoch # [5/10], batch # 10 with mini batch accuracy of 72.000% and loss of 0.994
Epoch # [5/10], batch # 20 with mini batch accuracy of 69.000% and loss of 1.016
Epoch # [5/10], batch # 30 with mini batch accuracy of 68.000% and loss of 0.979
Epoch # [5/10], batch # 40 with mini batch accuracy of 69.000% and loss of 1.160
Epoch # [5/10], batch # 50 with mini batch accuracy of 73.000% and loss of 0.870
Epoch # [5/10], batch # 60 with mini batch accuracy of 63.000% and loss of 1.263
Epoc

Epoch # [6/10], batch # 430 with mini batch accuracy of 66.000% and loss of 1.113
Epoch # [6/10], batch # 440 with mini batch accuracy of 67.000% and loss of 1.108
Epoch # [6/10], batch # 450 with mini batch accuracy of 66.000% and loss of 1.059
Epoch # [6/10], batch # 460 with mini batch accuracy of 67.000% and loss of 1.000
Epoch # [6/10], batch # 470 with mini batch accuracy of 64.000% and loss of 1.122
Epoch # [6/10], batch # 480 with mini batch accuracy of 68.000% and loss of 1.051
Epoch # [6/10], batch # 490 with mini batch accuracy of 77.000% and loss of 1.231
Epoch # [6/10], batch # 500 with mini batch accuracy of 61.000% and loss of 1.230
Epoch # [6/10] with overall accuracy of 70.070%
Test result of epoch [6/10] has accuracy of 57.750%
Epoch # [7/10], batch # 10 with mini batch accuracy of 77.000% and loss of 0.835
Epoch # [7/10], batch # 20 with mini batch accuracy of 67.000% and loss of 1.139
Epoch # [7/10], batch # 30 with mini batch accuracy of 73.000% and loss of 0.808
E

Epoch # [8/10], batch # 400 with mini batch accuracy of 71.000% and loss of 1.022
Epoch # [8/10], batch # 410 with mini batch accuracy of 69.000% and loss of 1.075
Epoch # [8/10], batch # 420 with mini batch accuracy of 70.000% and loss of 1.056
Epoch # [8/10], batch # 430 with mini batch accuracy of 70.000% and loss of 1.065
Epoch # [8/10], batch # 440 with mini batch accuracy of 64.000% and loss of 1.088
Epoch # [8/10], batch # 450 with mini batch accuracy of 73.000% and loss of 1.009
Epoch # [8/10], batch # 460 with mini batch accuracy of 66.000% and loss of 1.255
Epoch # [8/10], batch # 470 with mini batch accuracy of 71.000% and loss of 0.905
Epoch # [8/10], batch # 480 with mini batch accuracy of 67.000% and loss of 1.007
Epoch # [8/10], batch # 490 with mini batch accuracy of 72.000% and loss of 1.091
Epoch # [8/10], batch # 500 with mini batch accuracy of 74.000% and loss of 0.920
Epoch # [8/10] with overall accuracy of 70.884%
Test result of epoch [8/10] has accuracy of 57.610

Epoch # [10/10], batch # 370 with mini batch accuracy of 65.000% and loss of 1.052
Epoch # [10/10], batch # 380 with mini batch accuracy of 70.000% and loss of 1.134
Epoch # [10/10], batch # 390 with mini batch accuracy of 79.000% and loss of 0.969
Epoch # [10/10], batch # 400 with mini batch accuracy of 60.000% and loss of 1.260
Epoch # [10/10], batch # 410 with mini batch accuracy of 67.000% and loss of 0.983
Epoch # [10/10], batch # 420 with mini batch accuracy of 75.000% and loss of 0.997
Epoch # [10/10], batch # 430 with mini batch accuracy of 71.000% and loss of 0.904
Epoch # [10/10], batch # 440 with mini batch accuracy of 71.000% and loss of 1.003
Epoch # [10/10], batch # 450 with mini batch accuracy of 68.000% and loss of 1.087
Epoch # [10/10], batch # 460 with mini batch accuracy of 73.000% and loss of 1.006
Epoch # [10/10], batch # 470 with mini batch accuracy of 72.000% and loss of 1.035
Epoch # [10/10], batch # 480 with mini batch accuracy of 75.000% and loss of 0.893
Epoc

In [ ]:
# a = [1,3,4]
# np.argmax(a)

In [ ]:
###########use fastai to run###########

In [ ]:
##FASTAI INSTALLATION###

#conda install -c pytorch -c fastai fastai

In [ ]:
# from fastai.basic_data import DataBunch
# from fastai.train import Learner
# from fastai.metrics import accuracy

In [ ]:
# data = DataBunch.create(train_cifar,test_cifar,bs=50,path='./fastai/cifar100')
# learner = Learner(data,mymodel,loss_func=F.cross_entropy,metrics=[accuracy])
# learner.clip=0.1

In [ ]:
# learner.lr_find()

In [ ]:
# learner.fit_one_cycle(9,5e-3,wd=1e-4)